# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from meteostat import Point, Daily
pd.set_option('display.max_columns', None)

In [2]:
big_history = pd.read_csv('C:/Users/ryana/Downloads/constructiondata/big_history.csv', index_col=[0])

# Modeling

## Split data

The original plan was to use the last and second to last time slots for the testing and validation sets. However, I've opted instead for a traditional random split for a few reasons:
* While the 'answers' to the test set can technically show up in the training set (eg for a given place and time t, the time t+1 will contain data for how many crashes happened in time t and data for the nine times before t, which will match nine data points in t), I don't think the models will be able to make the connection betwen these variables, especially since the target variable is turned into a binary variable.
* Testing for predictions at only one time of year seems inadequate when seasonal conditions will probably have a significant impact.
* This allows for a larger test set.

In [3]:
from sklearn.preprocessing import StandardScaler

# The feature matrix is the last 30 rows (closure, temp, and precipitatoin data for 10 prior periods) scaled
x = big_history.iloc[:,4:].to_numpy()
scaler = StandardScaler()
x = scaler.fit_transform(x)

# The target is a binary value for whether there is a closure in each place/time
y = big_history.iloc[:,3]
def f(x):
    if(x>0):
        return 1
    else:
        return 0
y = y.apply(lambda x: f(x)).to_numpy()


In [4]:
# First take out the test set that will be used to calculate the generaliztion error 
from sklearn.model_selection import train_test_split
x_train_test, x_gen, y_train_test, y_gen = train_test_split(x, y, test_size=0.05, random_state=42)

In [5]:
# Then split the training and test sets
x_train, x_test, y_train, y_test = train_test_split(x_train_test, y_train_test, test_size=0.2, random_state=42)

## Train Models

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [7]:
def scores(y_test, y_pred):
    print('There were '+str(y_test.sum()) + ' postitive cases (' + str(round((y_test.sum()/len(y_test)),2)) + 
          ' of test set). The model predicted ' + str(y_pred.sum()) + 
          ' positive cases (' + str(round((y_pred.sum()/len(y_pred)),2)) + ' of test set).')
    print('Accuracy score: ' + str(round(accuracy_score(y_test, y_pred),2)))
    print('Recall score: ' + str(round(recall_score(y_test, y_pred),2)))
    print('Precision score: ' + str(round(precision_score(y_test, y_pred),2)))

### Logistic Regression

In [62]:
from sklearn.linear_model import LogisticRegression

In [63]:
log_params = {'solver':['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga'],
              'C':[1000, 100, 10, 1.0, 0.1, 0.001]}
log_grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid=log_params, scoring='recall', cv=3)
log_grid.fit(x_train, y_train)
log_grid.best_params_

{'C': 1000, 'solver': 'newton-cg'}

In [122]:
log_params2 = {'solver':['newton-cg'],
              'C':[200, 500, 800, 1000, 1200, 1500, 10000]}
log_grid2 = GridSearchCV(LogisticRegression(max_iter=1000), param_grid=log_params2, scoring='recall', cv=3)
log_grid2.fit(x_train, y_train)
log_grid2.best_params_

{'C': 200, 'solver': 'newton-cg'}

In [123]:
log = LogisticRegression(**log_grid2.best_params_)
log.fit(x_train, y_train)
log_pred = log.predict(x_test)

In [124]:
scores(y_test, log_pred)

There were 14585 postitive cases (0.28 of test set). The model predicted 7321 positive cases (0.14 of test set).
Accuracy score: 0.83
Recall score: 0.45
Precision score: 0.89


### Support Vector Machine

In [66]:
from sklearn.svm import LinearSVC

In [67]:
svc_params = {'C':[0.1, 1, 10, 100], 
              'class_weight':['balanced', None]}
svc_grid = GridSearchCV(LinearSVC(dual=False), param_grid=svc_params, scoring='recall', cv=3)
svc_grid.fit(x_train, y_train)
svc_grid.best_params_

{'C': 1, 'class_weight': 'balanced'}

In [105]:
svc_params2 = {'C':[0.5, 0.8, 1, 2, 4, 8], 
              'class_weight':['balanced']}
svc_grid2 = GridSearchCV(LinearSVC(dual=False), param_grid=svc_params2, scoring='recall', cv=3)
svc_grid2.fit(x_train, y_train)
svc_grid2.best_params_

{'C': 2, 'class_weight': 'balanced'}

In [119]:
svc_params3 = {'C':[1.5, 2, 2.5, 3, 3.5], 
              'class_weight':['balanced']}
svc_grid3 = GridSearchCV(LinearSVC(dual=False), param_grid=svc_params3, scoring='recall', cv=3)
svc_grid3.fit(x_train, y_train)
svc_grid3.best_params_

{'C': 2, 'class_weight': 'balanced'}

In [120]:
svc = LinearSVC(**svc_grid3.best_params_)
svc.fit(x_train, y_train)
svc_pred = svc.predict(x_test)

In [121]:
scores(y_test, svc_pred)

There were 14585 postitive cases (0.28 of test set). The model predicted 9123 positive cases (0.17 of test set).
Accuracy score: 0.84
Recall score: 0.52
Precision score: 0.84


### Random Forests

In [149]:
from sklearn.ensemble import RandomForestClassifier

In [69]:
rfc_params = {'n_estimators': [100, 500, 1000],
              'min_samples_split':[5,10,20],
              'max_depth': [2,4,8]}
rfc_grid = GridSearchCV(RandomForestClassifier(), param_grid=rfc_params, cv=3, scoring='recall')
rfc_grid.fit(x_train, y_train)
rfc_grid.best_params_

{'max_depth': 8, 'min_samples_split': 10, 'n_estimators': 100}

In [73]:
rfc_params2 = {'n_estimators': [50, 100, 200],
              'min_samples_split':[7,10,15],
              'max_depth': [6, 8, 16]}
rfc_grid2 = GridSearchCV(RandomForestClassifier(), param_grid=rfc_params2, cv=3, scoring='recall')
rfc_grid2.fit(x_train, y_train)
rfc_grid2.best_params_

{'max_depth': 16, 'min_samples_split': 7, 'n_estimators': 200}

In [106]:
rfc_params3 = {'n_estimators': [200, 300, 400],
              'min_samples_split':[7, 8, 9],
              'max_depth': [12, 16, 32]}
rfc_grid3 = GridSearchCV(RandomForestClassifier(), param_grid=rfc_params3, cv=3, scoring='recall')
rfc_grid3.fit(x_train, y_train)
rfc_grid3.best_params_

{'max_depth': 32, 'min_samples_split': 8, 'n_estimators': 200}

In [113]:
rfc = RandomForestClassifier(**rfc_grid3.best_params_)
rfc.fit(x_train, y_train)
rfc_pred = rfc.predict(x_test)

In [114]:
scores(y_test, rfc_pred)

There were 14585 postitive cases (0.28 of test set). The model predicted 11954 positive cases (0.23 of test set).
Accuracy score: 0.85
Recall score: 0.65
Precision score: 0.79


### Gradient Boosting Classifier

In [75]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [102]:
gbc_params = {'max_iter':[100, 500, 1000],
              'max_depth':[2,4,8], 
              'learning_rate':[0.01,0.03,0.3,1]}
gbc_grid = GridSearchCV(HistGradientBoostingClassifier(), param_grid=gbc_params, cv=3, scoring='recall')
gbc_grid.fit(x_train, y_train)
gbc_grid.best_params_

{'learning_rate': 1, 'max_depth': 2, 'max_iter': 500}

In [108]:
gbc_params2 = {'max_iter':[300, 500, 700],
              'max_depth':[1,2,3], 
              'learning_rate':[0.5,0.8,1]}
gbc_grid2 = GridSearchCV(HistGradientBoostingClassifier(), param_grid=gbc_params2, cv=3, scoring='recall')
gbc_grid2.fit(x_train, y_train)
gbc_grid2.best_params_

{'learning_rate': 0.5, 'max_depth': 3, 'max_iter': 300}

In [115]:
gbc_params3 = {'max_iter':[200, 300, 400],
              'max_depth':[3], 
              'learning_rate':[0.4, 0.5, 0.6, 0.7]}
gbc_grid3 = GridSearchCV(HistGradientBoostingClassifier(), param_grid=gbc_params3, cv=3, scoring='recall')
gbc_grid3.fit(x_train, y_train)
gbc_grid3.best_params_

{'learning_rate': 0.6, 'max_depth': 3, 'max_iter': 200}

In [117]:
gbc = HistGradientBoostingClassifier(**gbc_grid3.best_params_)
gbc.fit(x_train, y_train)
gbc_pred = gbc.predict(x_test) 

In [118]:
scores(y_test, gbc_pred)

There were 14585 postitive cases (0.28 of test set). The model predicted 12126 positive cases (0.23 of test set).
Accuracy score: 0.85
Recall score: 0.65
Precision score: 0.78


### K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [72]:
knn_params = {'n_neighbors':[3,5,10,20],
              'leaf_size':[15,30,50],
              'weights':['uniform','distance'],}
knn_grid = GridSearchCV(KNeighborsClassifier(), param_grid=knn_params, cv=3, scoring='recall')
knn_grid.fit(x_train, y_train)
knn_grid.best_params_

{'leaf_size': 15, 'n_neighbors': 3, 'weights': 'distance'}

In [109]:
knn_params2 = {'n_neighbors':[1,2,3,4],
              'leaf_size':[5,10,15,20,25],
              'weights':['distance'],}
knn_grid2 = GridSearchCV(KNeighborsClassifier(), param_grid=knn_params2, cv=3, scoring='recall')
knn_grid2.fit(x_train, y_train)
knn_grid2.best_params_

{'leaf_size': 5, 'n_neighbors': 1, 'weights': 'distance'}

In [116]:
knn_params3 = {'n_neighbors':[1],
              'leaf_size':range(1,10),
              'weights':['distance'],}
knn_grid3 = GridSearchCV(KNeighborsClassifier(), param_grid=knn_params3, cv=3, scoring='recall')
knn_grid3.fit(x_train, y_train)
knn_grid3.best_params_

{'leaf_size': 1, 'n_neighbors': 1, 'weights': 'distance'}

In [125]:
knn = KNeighborsClassifier(**knn_grid3.best_params_)
knn.fit(x_train, y_train)
knn_pred = knn.predict(x_test)

In [126]:
scores(y_test, knn_pred)

There were 14585 postitive cases (0.28 of test set). The model predicted 12231 positive cases (0.23 of test set).
Accuracy score: 0.79
Recall score: 0.55
Precision score: 0.65


###  Artificial Neural Network

In [9]:
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier, KerasRegressor

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [10]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[30]):
    model = keras.models.Sequential()
    options = {"input_shape": input_shape}
    
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu", **options))
        options = {}
    
    model.add(keras.layers.Dense(1, activation='sigmoid', **options))
    
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [11]:
keras_class = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=build_model)

In [130]:
ann_params = {"n_hidden":[0, 1, 2, 3],
             "n_neurons":np.arange(1, 100),
             "learning_rate":reciprocal(3e-4, 3e-2)}

rnd_search_cv = RandomizedSearchCV(keras_class, param_distributions=ann_params, n_iter=10, cv=3)
rnd_search_cv.fit(x_train, y_train, epochs=30, validation_split=0.1,
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/30
3938/3938 [==============================] - 8s 2ms/step - loss: 0.5122 - accuracy: 0.7707 - val_loss: 0.4793 - val_accuracy: 0.7917
Epoch 2/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.4840 - accuracy: 0.7921 - val_loss: 0.4673 - val_accuracy: 0.8012
Epoch 3/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.4775 - accuracy: 0.7980 - val_loss: 0.4613 - val_accuracy: 0.8056
Epoch 4/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.4737 - accuracy: 0.8020 - val_loss: 0.4572 - val_accuracy: 0.8062
Epoch 5/30
3938/3938 [==============================] - 6s 2ms/step - loss: 0.4712 - accuracy: 0.8039 - val_loss: 0.4544 - val_accuracy: 0.8092
Epoch 6/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.4692 - accuracy: 0.8058 - val_loss: 0.4517 - val_accuracy: 0.8104
Epoch 7/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.4676 - accuracy: 0.8068 - val_loss: 0.4513 - val_accuracy:

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001DED72DC730>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001DE947B0970>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [131]:
rnd_search_cv.best_params_

{'learning_rate': 0.005147516578588746, 'n_hidden': 1, 'n_neurons': 49}

In [143]:
ann_params2 = {"n_hidden":[0,1,2],
             "n_neurons":np.arange(25, 75),
             "learning_rate":reciprocal(1e-3, 1e-2)} 

rnd_search_cv2 = RandomizedSearchCV(keras_class, param_distributions=ann_params2, n_iter=10, cv=3)
rnd_search_cv2.fit(x_train, y_train, epochs=30, validation_split=0.1,
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/30
3938/3938 [==============================] - 8s 2ms/step - loss: 0.5632 - accuracy: 0.7424 - val_loss: 0.5387 - val_accuracy: 0.7568
Epoch 2/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.5275 - accuracy: 0.7610 - val_loss: 0.5215 - val_accuracy: 0.7667
Epoch 3/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.5130 - accuracy: 0.7691 - val_loss: 0.5106 - val_accuracy: 0.7735
Epoch 4/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.5025 - accuracy: 0.7755 - val_loss: 0.5020 - val_accuracy: 0.7810
Epoch 5/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.4940 - accuracy: 0.7811 - val_loss: 0.4950 - val_accuracy: 0.7886
Epoch 6/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.4867 - accuracy: 0.7863 - val_loss: 0.4889 - val_accuracy: 0.7916
Epoch 7/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.4803 - accuracy: 0.7905 - val_loss: 0.4834 - val_accuracy:

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x0000018F8A2C0310>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000018F8A2BF490>,
                                        'n_hidden': [0, 1, 2],
                                        'n_neurons': array([25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
       42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58,
       59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74])})

In [144]:
rnd_search_cv2.best_params_

{'learning_rate': 0.006177195956768579, 'n_hidden': 2, 'n_neurons': 28}

In [ ]:
ann_params3 = {"n_hidden":[1,2],
             "n_neurons":np.arange(25,50),
             "learning_rate":reciprocal(4e-3, 8e-3)}

rnd_search_cv3 = RandomizedSearchCV(keras_class, param_distributions=ann_params3, n_iter=10, cv=3)
rnd_search_cv3.fit(x_train, y_train, epochs=30, validation_split=0.1,
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/30
3938/3938 [==============================] - 8s 2ms/step - loss: 0.5245 - accuracy: 0.7637 - val_loss: 0.4902 - val_accuracy: 0.7912
Epoch 2/30
3938/3938 [==============================] - 8s 2ms/step - loss: 0.4613 - accuracy: 0.8045 - val_loss: 0.4513 - val_accuracy: 0.8135
Epoch 3/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.4285 - accuracy: 0.8227 - val_loss: 0.4276 - val_accuracy: 0.8226
Epoch 4/30
3938/3938 [==============================] - 8s 2ms/step - loss: 0.4093 - accuracy: 0.8313 - val_loss: 0.4112 - val_accuracy: 0.8304
Epoch 5/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.3979 - accuracy: 0.8361 - val_loss: 0.4012 - val_accuracy: 0.8352
Epoch 6/30
3938/3938 [==============================] - 8s 2ms/step - loss: 0.3915 - accuracy: 0.8384 - val_loss: 0.3991 - val_accuracy: 0.8350
Epoch 7/30
3938/3938 [==============================] - 7s 2ms/step - loss: 0.3872 - accuracy: 0.8404 - val_loss: 0.3951 - val_accuracy:

In [ ]:
ann = KerasClassifier(rnd_search_cv3.best_estimator_.model)

In [12]:
ann = KerasClassifier(build_model(learning_rate=0.006177195956768579, n_hidden=2, n_neurons=28))

In [13]:
ann.fit(x_train, y_train)
ann_pred = ann.predict(x_test)

1641/1641 [==============================] - 2s 1ms/step


In [14]:
scores(y_test, ann_pred)

There were 14617 postitive cases (0.28 of test set). The model predicted 6054 positive cases (0.12 of test set).
Accuracy score: 0.81
Recall score: 0.36
Precision score: 0.88


## Final Model Selection

In [15]:
pd.DataFrame([[0.83,0.45,0.89],[0.84,0.52,0.84],[0.85,0.65,0.79],
              [0.85,0.65,0.78],[0.79,0.55,0.65],[0.81,0.36,0.88]], 
             columns = ['Accuracy', 'Recall', 'Precision'], 
             index=['Logistic Regression', 'Support Vector Machine','Random Forests', 
                    'Gradient Boosting Classifier','K-Nearest Neighbors', 'Artificial Neural Net'])


,Accuracy,Recall,Precision
Logistic Regression,0.83,0.45,0.89
Support Vector Machine,0.84,0.52,0.84
Random Forests,0.85,0.65,0.79
Gradient Boosting Classifier,0.85,0.65,0.78
K-Nearest Neighbors,0.79,0.55,0.65
Artificial Neural Net,0.81,0.36,0.88


The Random Forest Classifier and Gradient Boosting Classifier are the best models, showing significantly higher recall (and slightly lower precision) than Logistic Regression and SVM and totally outperforming K-Nearest Neighbors and the artificial neural net. Random Forests is slightly better so it will be the final model.

In [151]:
#fit the best model on all the data besides the generalization error set
rfc.fit(x_train_test, y_train_test)

RandomForestClassifier(max_depth=32, min_samples_split=8, n_estimators=200)

In [152]:
#test for generalization error
final_pred = rfc.predict(x_gen)
scores(y_gen, final_pred)

There were 3871 postitive cases (0.28 of test set). The model predicted 3163 positive cases (0.23 of test set).
Accuracy score: 0.86
Recall score: 0.65
Precision score: 0.8


In [153]:
#export the model as a .pkl file
import joblib
joblib.dump(rfc, 'C://Users//ryana//Downloads//flask_aws_materials//flask_aws_materials//finalModel.pkl', compress = 1)

['C://Users//ryana//Downloads//flask_aws_materials//flask_aws_materials//finalModel.pkl']